In [1]:
import os, subprocess
import json
import uproot
import awkward as ak
import numpy as np
from coffea import processor, util, hist

%matplotlib inline
from matplotlib import lines as mlines
import matplotlib.pyplot as plt
from cycler import cycler

In [2]:
lumis = {}
lumis['2016'] = 35.9
lumis['2017'] = 41.1
lumis['2018'] = 59.9

nfiles_mc = {}
nfiles_mc['2016'] = 113
nfiles_mc['2017'] = 167
nfiles_mc['2018'] = 201

with open('xsec.json') as f:
  xs = json.load(f)

with open('pmap.json') as f:
  pmap = json.load(f)

In [3]:
year = '2017'
outsum = processor.dict_accumulator()

In [4]:
# Load all MC
for n in range(1,nfiles_mc[year]+1):
    print(n)
    filename = 'condor/outfiles/'+year+'_'+str(n)+'.coffea'
    if os.path.isfile(filename):
        out = util.load(filename)
        outsum.add(out)
    else:
        print("File " + filename + " is missing")
        
scale_lumi = {k: xs[k] * 1000 *lumis[year] / w for k, w in outsum['sumw'].items()}
outsum['cutflow'].scale(scale_lumi, 'dataset')

1
2
3
4
5
6
7
8
9
10
11
12
13
File condor/outfiles/2017_13.coffea is missing
14
15
File condor/outfiles/2017_15.coffea is missing
16
File condor/outfiles/2017_16.coffea is missing
17
File condor/outfiles/2017_17.coffea is missing
18
File condor/outfiles/2017_18.coffea is missing
19
File condor/outfiles/2017_19.coffea is missing
20
File condor/outfiles/2017_20.coffea is missing
21
File condor/outfiles/2017_21.coffea is missing
22
File condor/outfiles/2017_22.coffea is missing
23
24
File condor/outfiles/2017_24.coffea is missing
25
26
27
28
29
30
File condor/outfiles/2017_30.coffea is missing
31
File condor/outfiles/2017_31.coffea is missing
32
File condor/outfiles/2017_32.coffea is missing
33
File condor/outfiles/2017_33.coffea is missing
34
File condor/outfiles/2017_34.coffea is missing
35
36
37
38
File condor/outfiles/2017_38.coffea is missing
39
40
File condor/outfiles/2017_40.coffea is missing
41
42
43
44
45
46
47
File condor/outfiles/2017_47.coffea is missing
48
File condor/outfile

In [5]:
templates = outsum['cutflow'].group('dataset', hist.Cat('process', 'Process'), pmap)

/uscms_data/d3/jennetd/hbb-prod-modes/february-2021/inclusive/coffeaenv/lib/python3.7/site-packages/coffea/hist/hist_tools.py:347: RuntimeWarning: Not all requested indices present in <Cat (name=dataset) instance at 0x7fb3a92e6750>
  warnings.warn("Not all requested indices present in %r" % self, RuntimeWarning)


In [6]:
cutflow = templates.sum('genflavor').integrate('region','signal')

In [7]:
cutflow.values()

{('ZH',): array([1.98852207e+04, 1.98852207e+04, 8.34769019e+01, 7.79854123e+01,
        7.79854123e+01, 4.56652251e+01, 2.24968474e+01, 1.64864151e+01,
        1.17847395e+01, 4.73138589e+00, 0.00000000e+00]),
 ('WH',): array([7.86509728e+03, 7.86509728e+03, 2.80829382e+01, 2.56981307e+01,
        2.56981307e+01, 1.54672639e+01, 6.82735015e+00, 6.47307119e+00,
        4.85319605e+00, 1.95378632e+00, 0.00000000e+00]),
 ('QCD',): array([3.48032887e+07, 3.48032887e+07, 1.33970105e+07, 1.13321569e+07,
        1.13321569e+07, 2.84696504e+06, 2.62893581e+06, 2.57618135e+06,
        2.23474727e+06, 3.33287161e+04, 0.00000000e+00]),
 ('VV',): array([4.86367163e+06, 4.86367163e+06, 5.33244331e+03, 4.90298366e+03,
        4.90298366e+03, 2.84779438e+03, 2.59117625e+03, 1.91288114e+03,
        1.20003194e+03, 5.48382316e+01, 0.00000000e+00]),
 ('Wjets',): array([8.95208116e+06, 8.95208116e+06, 3.09004330e+05, 2.65881742e+05,
        2.65881742e+05, 1.11223092e+05, 1.03167017e+05, 1.00375152e+05,

In [8]:
import pandas as pd

In [9]:
df1 = pd.DataFrame([])

df1['ggF'] = cutflow.values()[('ggF',)]
df1['VBF'] = cutflow.values()[('VBF',)]
df1['WH'] = cutflow.values()[('WH',)]
df1['ZH'] = cutflow.values()[('ZH',)]
df1['ttH'] = cutflow.values()[('ttH',)]

df1 = df1[3:-1].astype('int')
df1.index = ['Jet kinematics','Jet ID','Jet $N_2^\text{DDT}$','Opp. hem. b veto','MET $<$ 140 GeV','No leptons','DeepDoubleB']

df1.to_latex(buf=year+'/cutflow-sig.tex')

KeyError: ('ggF',)

In [ ]:
df2 = pd.DataFrame([])

df2['QCD'] = cutflow.values()[('QCD',)]
df2['Wjets'] = cutflow.values()[('Wjets',)]
df2['Zjets'] = cutflow.values()[('Zjets',)]
df2['VV'] = cutflow.values()[('VV',)]
df2['ttbar'] = cutflow.values()[('ttbar',)]
df2['singlet'] = cutflow.values()[('singlet',)]

df2 = df2[3:-1].astype('int')
df2.index = ['Jet kinematics','Jet ID','Jet $N_2^\text{DDT}$','Opp. hem. b veto','MET $<$ 140 GeV','No leptons','DeepDoubleB']

df2.to_latex(buf=year+'/cutflow-bkg.tex')